<a href="https://colab.research.google.com/github/AraiKensuke/AIiRPS/blob/master/Launch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import importlib

if importlib.util.find_spec("AIiRPS") is None:
  !git clone https://github.com/AraiKensuke/AIiRPS.git
if importlib.util.find_spec("pyPG") is None:
  ###  First, clone the pyPG package from Github.  This will only last for this runtime session.  Do this every time you re-open notebook.
  !git clone https://github.com/AraiKensuke/pyPG.git
  %cd /content/pyPG/

  ###  pyPG is written in C++/python, and uses the C library GNU Scientific Library.  
  !sudo apt-get install libgsl-dev   

  ###  setup and build pyPG
  !python setup.py build_ext --inplace

  %cd /content   

if importlib.util.find_spec("LOSTtmp") is None:
  ###  LOST  (FFBS algorithm implemented here)
  !git clone https://github.com/AraiKensuke/LOSTtmp.git

  %cd /content/LOSTtmp/
  !python setup.py build_ext --inplace
  %cd /content

import AIiRPS
import pyPG
import LOSTtmp

Cloning into 'pyPG'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.
/content/pyPG
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsl23 libgslcblas0
Suggested packages:
  gsl-ref-psdoc | gsl-doc-pdf | gsl-doc-info | gsl-ref-html
The following NEW packages will be installed:
  libgsl-dev libgsl23 libgslcblas0
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,926 kB of archives.
After this operation, 9,474 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgslcblas0 amd64 2.4+dfsg-6 [79.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl23 amd64 2.4+dfsg-6 [823 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgsl-dev amd64 2.4+dfsg-6 [1,023 kB]
Fetched 1,926 kB in 1s 

In [3]:
#!/usr/bin/python

import numpy as _N  #  must import this before pyPG when running from shell
import pickle

import AIiRPS.utils.read_taisen as _rd
import matplotlib.pyplot as _plt
import os
import sys
import AIiRPS.models.labels as labels
import AIiRPS.models.multinomial_gibbs as _mg

from AIiRPS.utils.dir_util import getResultFN

def union_arrs(a1, a2):
     return _N.sort(_N.array(a1.tolist() + a2.tolist()))

def fill_unobserved(arr):
    #  we assume old value is kept until new observation
    nz = _N.where(arr != -100)[0]
    iLastObs = arr[0] if (arr[0] != -100) else _N.mean(arr[nz])
    for i in range(1, arr.shape[0]):
        arr[i] = arr[i] if (arr[i] != -100) else iLastObs
        if arr[i] != -100:
            iLastObs = arr[i]
    
def depickle(s):
     import pickle
     with open(s, "rb") as f:
          lm = pickle.load(f)
     return lm

###
#   Here the latent state is B1, B2.
#   Probability of going to stay, weaker, stronger for just the WIN condition
#   

#  input data
#  m_hands    # machine hands  size N         1 -1  1
#  h_hands    # human hands    size N        -1 -1  1
#  -1 -1  1  1 -1  1 
#      0  1  0  1  1

fns     = ["ME,WTL", "ME,RPS", "AI,WTL", "AI,RPS", "OU,WTL", "OU,RPS"]

fnt_tck = 10
fnt_lbl = 12

fnt_tck = 15
fnt_lab = 16

_WTL = 10
_ME_WTL = 0
_RELATIVE_LAST_ME = 1    #  what to use as a comparison for when player switches

smp_every = 50
ITER = 30000
it0  = 28000
it1  = 30000

label=6
#  priors for inv gamma   #  B/(a+1)

rndmz = False

######  AR(1) coefficient range 
a_F0      = -1;    b_F0      =  1

######  int main 
#int i,pred,m,v[3],x[3*N+1],w[9*N+3],fw[3];

know_gt  = False
signal   = _RELATIVE_LAST_ME
covariates = _WTL
sig_cov   = _ME_WTL

tr0      = 0
tr1      = -1

#####USE real
#dat_fn        = "20Aug18-1624-01"

#dat_fn="20Jun01-0748-03"
#dat_fn="20May29-1923-44"
#dat_fn="20May29-1419-14"
#
#dat_fn      = "20Jan09-1504-32"
#dat_fn="20Aug12-1331-06"
#dat_fn="20Jan08-1703-13"
#dat_fn="20Aug12-1252-50"
#dat_fn="20Nov22-1108-25"
dat_fn="20Nov21-1959-30"

random_walk = True
flip_human_AI=False
#########################################
a_q2, B_q2 = labels.get_a_B(label)

scov = "WTL"
ssig = "ME"

if tr1 < 0:
    tr1 = None
sran   = ""

s_flip = "_flip" if flip_human_AI else ""
if not know_gt: 
    _hnd_dat = _rd.return_hnd_dat(dat_fn, flip_human_AI=flip_human_AI)    
else:
    _hnd_dat     = _N.loadtxt("/Users/arai/nctc/Workspace/AIiRPS_SimDAT/rpsm_%s.dat" % dat_fn, dtype=_N.int)
    lmGT = depickle("/Users/arai/nctc/Workspace/AIiRPS_SimDAT/rpsm_%s.dmp" % dat_fn)
    Ts = lmGT["Ts_timeseries"]


ModuleNotFoundError: ignored

In [5]:
import LOSTtmp.kfARlib1c

ModuleNotFoundError: ignored

In [6]:
!ls

AIiRPS	LOSTtmp  pyPG  sample_data


In [8]:
!ls LOSTtmp/LOSTtmp

__init__.py  kfARlib1c.cpython-36m-x86_64-linux-gnu.so
kfARlib1c.c  kfARlib1c.pyx


In [9]:
import pyPG